# Clustering Visualization
* StellarAlgo Data Science
* Peter Morrison
* June 24, 2022

## Loading Data and Creating the Model
We will load up an example dataset to just get some simple clustering results to demonstrate with.

In [8]:
import getpass
import pandas as pd
import pyodbc
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pycaret.clustering import *

In [3]:
# connect to SQL Server.
SERVER = '52.44.171.130' 
DATABASE = 'datascience' 
USERNAME = 'dsAdminWrite' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [4]:
TEAMS = [
    {
            "mssql_dbname": "stlrMLS",
            "lkupclientid": "31",
            "clientcode": "sacfc",
            "train_year": 2021,
            "test_year": 2022
        }
]

In [5]:
print(f"GETTING TEAM DATASETS:")

team_datasets = []
for team in TEAMS:

    cursor = CNXN.cursor()

    storedProc = (
        f"""Exec {team["mssql_dbname"]}.[ds].[getRetentionScoringModelData] {team["lkupclientid"]}"""
    )

    df = pd.read_sql(storedProc, CNXN)

    df["year"] = pd.to_numeric(df["year"])
    df = df[df["year"] <= team["train_year"]]

    print(f" > ADDING TEAM TO DATASET: {team['clientcode']}")

    CNXN.commit()
    cursor.close()

    team_datasets.append(df)

print(f"TOTAL TEAMS IN DATASET: {len(team_datasets)}")

GETTING TEAM DATASETS:


/Users/petermorrison/opt/miniconda3/envs/package/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


 > ADDING TEAM TO DATASET: sacfc
TOTAL TEAMS IN DATASET: 1


In [6]:
df_dataset = pd.concat(team_datasets)

print(df_dataset.shape)
print(df_dataset.year.value_counts())

(10409, 53)
2017    2358
2018    2202
2021    2078
2020    1988
2019    1767
2016      16
Name: year, dtype: int64


In [9]:
df.head()

,lkupClientId,dimCustomerMasterId,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,31,441583734,2016,Mini/Flex Plan,1110.0,2016-09-17,0.888889,11,TRUE,2920,926,214.287098,3,0,2,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,2,0,0,F,1,1,None,None,None,None,0.0,0,17,None,0
1,31,441547723,2016,Mini/Flex Plan,350.0,2016-09-17,0.000000,16,TRUE,2190,16,214.287098,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,17,None,0
2,31,441585328,2016,Mini/Flex Plan,45.0,2016-09-03,0.500000,11,TRUE,2190,32,214.287098,1,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,16.0,0,17,None,0
3,31,441555341,2016,Mini/Flex Plan,171.0,2016-09-17,1.000000,2,TRUE,2190,23,214.287098,2,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,4,0,0,0,2,0,0,M,1,1,None,None,None,None,0.0,0,17,None,0
4,31,441615319,2016,Mini/Flex Plan,225.0,2016-09-17,0.916667,19,TRUE,2555,61,214.287098,4,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,2,0,0,Unknown,1,1,None,None,None,None,0.0,0,17,None,0


## Creating Model
In this section we make the model with Pycaret

In [16]:
cluster = setup(df_dataset, session_id = 7652)

TypeError: setup() got an unexpected keyword argument 'data_split_shuffle'

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?